In [ ]:
%run utils.ipynb
%run log.ipynb
%run arctan.ipynb

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import seaborn as sns
import numpy as np
import random
import networkx as nx
from collections import namedtuple
from PIL import Image, ImageSequence

In [ ]:
class Visualizer:

    def __init__(self, name, simulator, iterations, seed, **kwargs):
        sns.set()
        self.name = name
        self.simulator = simulator
        self.iterations = iterations
        self.coords = []
        self.simulation = None
        self.seed = seed

        self.G = nx.Graph()

        self.fig = plt.figure(figsize=(5, 15))
        self.axes = [
            plt.subplot(4, 1, 1), 
            plt.subplot(4, 1, 2), 
            ]
        self.graph_ax = plt.subplot(4, 1,4)
        self.titles = ["Fitness", "Tau"]

    def simulate(self, num_nodes=50, eta_0=20, alpha=50, delta=0, rho=0.1, M=100):
        random.seed(self.seed)
        self.coords = [Point(random.uniform(0,100), random.uniform(0, 100)) for _ in range(num_nodes)]
        self.G.add_nodes_from(range(num_nodes))
        
        #random.seed(200)
        self.simulation = self.simulator(self.coords, eta_0=eta_0, alpha=alpha, delta=delta, rho=rho, M=M, stopping_iter=self.iterations)
    
    def draw_graph(self, n):
        self.graph_ax.clear()
        self.G.remove_edges_from(self.G.edges)

        dep = self.simulation.solution_history[n]
        dest = dep[1:] + [dep[0]]

        for a, b in zip(dep, dest):
            self.G.add_edge(a, b)

        nx.draw(self.G, self.coords, ax=self.graph_ax, node_size=10, alpha=.5)

    def visualize(self, label):

        Y = [
            self.simulation.fitness_list, 
            self.simulation.tau_history, 
            self.simulation.T_history
            ]
        
        x = np.arange(self.iterations)
        
        reducer = 50
        self.fig.tight_layout()
        def animate(n):
            res = []
            i = n * reducer

            for ax, y, t in zip(self.axes, Y, self.titles):
                ax.clear()
                ax.set_xlim(0, self.iterations)
                ax.set_ylim(min(0, min(y)) * 1.2, max(y)*1.2)        
                ax.set_title(t)

                line, = ax.plot(x[0:i], y[0:i], color='grey', lw=1)
                point, = ax.plot(x[i], y[i], color='grey', marker='.')

                res.append(line)
                res.append(point)
            self.draw_graph(i)
            
            return res
        
        
        ani = FuncAnimation(self.fig, animate, interval=10, blit=True, repeat=True, frames=self.iterations//reducer)
        # html = HTML(ani.to_jshtml())
        # with open(f"{self.name}_{label}.html", "w") as f:
        #     f.write(html.data)
        ani.save(f"{self.name}_{label}.gif", dpi=300, writer=PillowWriter(fps=24))
    
    def visualize_anneal(self, **kwargs):
        self.simulate(**kwargs)
        self.simulation.anneal()
        self.visualize('anneal')
    
    def visualize_IET_anneal(self, **kwargs):
        self.simulate(**kwargs)
        self.simulation.IET_anneal()
        self.visualize('IET-anneal')

In [ ]:
if __name__ == "__main__":

    RUN_SIM = True
    RUN_STITCH = True
    BAD_SEED = 6338
    GOOD_SEED = 9332

    arctan_params = {
        'eta_0': 25,
        'alpha': 50,
        'delta': 0,
        'rho': 10
    }
    
    log_params = {
        'eta_0': 50,
        'alpha': 100,
        'delta': 0,
        'rho': 1
    }

    if RUN_SIM:
        good_anneal = Visualizer("good_vanilla", logSim, 20000, seed=GOOD_SEED)
        #bad_anneal = Visualizer("bad_vanilla", logSim, 20000, seed=GOOD_SEED)
        
        good_anneal.visualize_anneal(**arctan_params)
        #bad_anneal.visualize_anneal(**arctan_params)

        good_log = Visualizer("good_log", logSim, 20000, seed=GOOD_SEED)
        #bad_log = Visualizer("bad_log", logSim, 20000, seed=BAD_SEED)

        good_log.visualize_IET_anneal(**log_params)
        #bad_log.visualize_IET_anneal(eta_0 = 50,alpha = 100,delta= 0 ,rho = 1,M = 100)

        #good_arctan = Visualizer("good_arctan", arctanSim, 20000, seed=GOOD_SEED)
        #bad_arctan = Visualizer("bad_arctan", arctanSim, 20000, seed=GOOD_SEED)

        #good_arctan.visualize_IET_anneal(**arctan_params)
        #bad_arctan.visualize_IET_anneal(**arctan_params)
    
    if RUN_STITCH:
        comparisons = [
            ('good_vanilla_anneal', 'good_log_IET-anneal'),
            #('bad_vanilla_anneal', 'bad_log_IET-anneal'),
            #('good_vanilla_anneal', 'good_arctan_IET-anneal'),
            #('bad_vanilla_anneal', 'bad_arctan_IET-anneal'),
            #('good_log_IET-anneal', 'good_arctan_IET-anneal'),
            #('bad_log_IET-anneal', 'bad_arctan_IET-anneal'),
            #('bad_log_IET-anneal', 'good_arctan_IET-anneal'),
            #('good_log_IET-anneal', 'bad_arctan_IET-anneal'),
        ]

        for a, b in comparisons:
            GIFStich(f"{a}.gif", f"{b}.gif", f"{a}v{b}.gif")

In [ ]:
good_arctan = Visualizer("good_arctan", arctanSim, 20000, seed=9332)
good_anneal = Visualizer("good_vanilla", logSim, 20000, seed=9332)

good_arctan.simulate(eta_0 = 25,alpha = 50,delta= 0 ,rho = 10)
good_anneal.simulate(eta_0 = 25,alpha = 50,delta= 0 ,rho = 10)

good_arctan.simulation.IET_anneal()
good_anneal.simulation.anneal()


(good_anneal.simulation.best_fitness - good_arctan.simulation.best_fitness)/good_anneal.simulation.best_fitness